In [54]:
import os, sklearn, math
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from joblib import dump, load

tf.random.set_seed(42)

data = '231008_0119_2kPa_2line_Data'
dataname = data+'_B18'
filename = 'myData/'+data+'.csv'
SD1 = pd.read_csv(filename)
SD1=SD1.iloc[:,:28]
SensorData1 = SD1
SD1.describe()
SDreal = SD1
SDreal.describe()

,Sx00,Sx01,Sx02,Sx03,Sx04,Sx05,Sx06,Sx07,Sx10,Sx11,...,Sx26,Sx27,Sx30,Sx31,Sx32,Sx33,Force,z,theta,r
count,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,...,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000
mean,47.789284,46.587351,55.058288,43.507158,51.653830,64.754176,60.831080,53.016065,62.286681,56.452334,...,47.492486,54.471498,50.516973,55.370615,55.781778,49.804991,39.021419,80.001453,170.042368,3.200062
std,1.435331,1.708339,1.499085,1.786158,1.199627,1.697183,1.966231,1.891453,1.799997,1.444436,...,2.180379,3.001172,2.486547,1.726292,1.757992,1.515384,30.667866,48.438790,103.739277,1.600254
min,41.300000,40.700000,47.700000,32.800000,43.300000,45.700000,46.800000,44.600000,50.700000,47.200000,...,37.500000,45.900000,42.700000,48.200000,46.400000,39.000000,-455.760000,0.000000,0.000000,0.800000
25%,46.900000,45.500000,54.100000,42.300000,50.900000,64.000000,60.000000,51.800000,61.200000,55.500000,...,46.300000,52.800000,49.100000,54.200000,54.700000,48.800000,11.070000,40.000000,80.000000,1.600000
50%,47.800000,46.600000,55.100000,43.500000,51.700000,64.900000,61.000000,52.900000,62.200000,56.500000,...,47.300000,54.000000,50.200000,55.300000,55.800000,49.800000,33.260000,80.000000,180.000000,3.200000
75%,48.700000,47.600000,56.100000,44.600000,52.400000,65.700000,62.000000,54.000000,63.200000,57.400000,...,48.300000,55.300000,51.400000,56.400000,56.900000,50.800000,59.360000,120.000000,260.000000,4.800000
max,54.500000,55.400000,61.900000,52.200000,58.000000,69.300000,66.400000,63.900000,71.100000,66.800000,...,63.400000,70.900000,66.700000,66.700000,65.700000,56.800000,134.500000,160.000000,340.000000,5.600000


In [55]:
# SDreal 값이 999이거나 100이상인 이상치 제거
SDreal = SDreal[~(SDreal == 999).any(axis=1)]
SDreal = SDreal[~(SDreal == 255).any(axis=1)]
SDreal = SDreal[~(SDreal == 4).any(axis=1)]

# SDreal에서 결측값 제거
SDreal = SDreal[~SDreal.isna().any(axis=1)]

# 'Force', 'theta', 'z', 'r' 열을 제외한 열에서 100 이상의 값을 제거
cols_to_exclude = ["Force", "theta", "z", "r"]
cols_to_check = [col for col in SDreal.columns if col not in cols_to_exclude]

SDreal = SDreal[~((SDreal < 0) & (SDreal.columns == "Force")).any(axis=1)]
SDreal = SDreal[~((SDreal >= 10000) & (SDreal.columns == "Force")).any(axis=1)]


#SDreal = SDreal[SDreal["r"] >= 0.8*1]
SDreal = SDreal[SDreal["z"] <= 160]
SDreal = SDreal[SDreal["z"] >= 0]

import numpy as np
toRad = np.pi/180.0
SDreal['cos']=np.cos(SDreal['theta']*toRad)
SDreal['sin']=np.sin(SDreal['theta']*toRad)
SDreal.describe()


,Sx00,Sx01,Sx02,Sx03,Sx04,Sx05,Sx06,Sx07,Sx10,Sx11,...,Sx30,Sx31,Sx32,Sx33,Force,z,theta,r,cos,sin
count,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,...,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,9.912200e+04
mean,47.785374,46.565754,55.053824,43.490999,51.653430,64.754710,60.839275,53.022690,62.273591,56.440344,...,50.504461,55.366824,55.774105,49.784367,37.789275,79.997902,170.048223,3.066750,-0.000322,-7.532672e-05
std,1.436813,1.708538,1.496517,1.783120,1.205012,1.686369,1.947581,1.893671,1.793562,1.451408,...,2.460390,1.731227,1.757833,1.522854,30.908831,48.439815,103.747361,1.692098,0.707167,7.070537e-01
min,41.300000,40.700000,47.700000,32.800000,43.300000,45.700000,46.800000,44.600000,50.700000,47.200000,...,42.700000,48.200000,46.400000,39.000000,4.010000,0.000000,0.000000,0.800000,-1.000000,-9.848078e-01
25%,46.800000,45.500000,54.100000,42.300000,50.900000,64.000000,60.000000,51.800000,61.200000,55.500000,...,49.100000,54.200000,54.600000,48.800000,8.650000,40.000000,80.000000,1.600000,-0.766044,-6.427876e-01
50%,47.800000,46.500000,55.100000,43.500000,51.700000,64.900000,61.000000,52.900000,62.100000,56.500000,...,50.200000,55.300000,55.800000,49.800000,29.830000,80.000000,180.000000,2.400000,-0.173648,1.224647e-16
75%,48.700000,47.600000,56.000000,44.600000,52.400000,65.700000,62.000000,54.000000,63.200000,57.400000,...,51.400000,56.400000,56.900000,50.800000,58.790000,120.000000,260.000000,4.800000,0.766044,6.427876e-01
max,54.500000,55.400000,61.900000,52.200000,58.000000,69.300000,66.400000,63.900000,71.100000,66.800000,...,64.900000,66.700000,65.700000,56.800000,134.500000,160.000000,340.000000,5.600000,1.000000,9.848078e-01


In [56]:
# Filtering the rows where Force is 0
force_zero_data = SDreal[SDreal['Force'] == 0]

# Calculating the mean values for each sensor when Force is 0
sensor_columns = SDreal.columns[SDreal.columns.str.startswith('Sx')]
mean_values = force_zero_data[sensor_columns].mean()

# Subtracting mean values from each sensor reading, squaring, and summing across rows
squared_differences = (SDreal[sensor_columns] - mean_values) ** 2
SDreal['Root_Sum_Square'] = squared_differences.sum(axis=1) ** 0.5

# Display the first few rows of the updated data
SDreal.describe()

,Sx00,Sx01,Sx02,Sx03,Sx04,Sx05,Sx06,Sx07,Sx10,Sx11,...,Sx31,Sx32,Sx33,Force,z,theta,r,cos,sin,Root_Sum_Square
count,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,...,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,9.912200e+04,99122.0
mean,47.785374,46.565754,55.053824,43.490999,51.653430,64.754710,60.839275,53.022690,62.273591,56.440344,...,55.366824,55.774105,49.784367,37.789275,79.997902,170.048223,3.066750,-0.000322,-7.532672e-05,0.0
std,1.436813,1.708538,1.496517,1.783120,1.205012,1.686369,1.947581,1.893671,1.793562,1.451408,...,1.731227,1.757833,1.522854,30.908831,48.439815,103.747361,1.692098,0.707167,7.070537e-01,0.0
min,41.300000,40.700000,47.700000,32.800000,43.300000,45.700000,46.800000,44.600000,50.700000,47.200000,...,48.200000,46.400000,39.000000,4.010000,0.000000,0.000000,0.800000,-1.000000,-9.848078e-01,0.0
25%,46.800000,45.500000,54.100000,42.300000,50.900000,64.000000,60.000000,51.800000,61.200000,55.500000,...,54.200000,54.600000,48.800000,8.650000,40.000000,80.000000,1.600000,-0.766044,-6.427876e-01,0.0
50%,47.800000,46.500000,55.100000,43.500000,51.700000,64.900000,61.000000,52.900000,62.100000,56.500000,...,55.300000,55.800000,49.800000,29.830000,80.000000,180.000000,2.400000,-0.173648,1.224647e-16,0.0
75%,48.700000,47.600000,56.000000,44.600000,52.400000,65.700000,62.000000,54.000000,63.200000,57.400000,...,56.400000,56.900000,50.800000,58.790000,120.000000,260.000000,4.800000,0.766044,6.427876e-01,0.0
max,54.500000,55.400000,61.900000,52.200000,58.000000,69.300000,66.400000,63.900000,71.100000,66.800000,...,66.700000,65.700000,56.800000,134.500000,160.000000,340.000000,5.600000,1.000000,9.848078e-01,0.0


In [44]:
root_sum_square_values = SDreal['Root_Sum_Square']
root_sum_square_values.describe()  # Displaying the first few values for brevity

count    15028.000000
mean        10.157357
std          2.606381
min          3.933562
25%          8.330323
50%          9.782536
75%         11.667390
max         22.545824
Name: Root_Sum_Square, dtype: float64

In [53]:
# Filtering the data where Root_Sum_Square is less than or equal to 10
data_below_10 = SDreal[SDreal['Root_Sum_Square'] <= 10]

# Displaying these filtered data rows
data_below_10.describe()  # Displaying the first few rows for brevity

,Sx00,Sx01,Sx02,Sx03,Sx04,Sx05,Sx06,Sx07,Sx10,Sx11,...,Sx31,Sx32,Sx33,Force,z,theta,r,cos,sin,Root_Sum_Square
count,8051.000000,8051.000000,8051.000000,8051.000000,8051.000000,8051.000000,8051.000000,8051.000000,8051.000000,8051.000000,...,8051.000000,8051.000000,8051.000000,8051.000000,8051.000000,8051.000000,8051.000000,8051.000000,8051.000000,8051.000000
mean,44.094465,44.410135,49.628610,44.946721,48.757907,61.667604,56.211556,47.649025,59.048527,54.857647,...,52.172844,51.265941,42.983811,29.824657,90.229537,132.731338,1.829139,0.038686,0.273372,8.254329
std,1.702179,2.251179,1.418498,1.645889,1.764665,1.553096,1.507395,2.102501,1.896052,1.357502,...,1.612368,1.620386,1.699350,23.312331,6.429975,94.981814,1.624153,0.700576,0.658087,1.196243
min,38.370000,37.770000,44.770000,40.050000,42.050000,52.960000,49.090000,40.990000,54.460000,50.330000,...,45.460000,45.340000,36.950000,0.000000,82.000000,0.000000,0.000000,-1.000000,-0.984808,3.933562
25%,42.910000,42.800000,48.630000,43.850000,48.100000,60.990000,55.200000,46.175000,57.870000,53.990000,...,51.270000,50.320000,41.920000,13.610000,82.000000,60.000000,0.800000,-0.766044,-0.342020,7.461543
50%,44.030000,44.060000,49.700000,44.830000,49.070000,61.840000,56.250000,47.350000,58.710000,54.950000,...,52.280000,51.420000,43.030000,28.800000,90.000000,120.000000,1.600000,0.173648,0.342020,8.439808
75%,45.190000,45.560000,50.590000,45.915000,49.880000,62.600000,57.250000,48.830000,59.670000,55.770000,...,53.280000,52.370000,44.130000,47.480000,98.000000,200.000000,2.400000,0.766044,0.866025,9.231962
max,49.200000,51.240000,53.980000,50.990000,53.090000,66.250000,61.210000,55.240000,66.350000,59.040000,...,56.590000,56.790000,47.760000,85.540000,98.000000,340.000000,5.600000,1.000000,0.984808,9.999500
